<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.5

tensorflow==2.5.0


In [3]:
# change these to try this notebook out
BUCKET = 'dev-xai-natality'
PROJECT = 'hack4ssb-team1'
REGION = 'europe-west3'

In [4]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [5]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://dev-xai-natality/...


<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [7]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-7170969733900686954,331274
1,5896567601480310696,342825
2,-5742197815970064689,359684
3,-774501970389208065,349134
4,454960867574323744,346515


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [8]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13396 examples in the train dataset and 3203 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [9]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.125340,False,23,1,31.0,3095933535584005890
1,5.249206,True,26,1,37.0,3095933535584005890
2,6.624891,False,25,1,37.0,3095933535584005890
3,6.867399,False,32,1,39.0,3095933535584005890
4,6.876218,True,20,1,39.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [10]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13387.000000,13396.000000,13396.000000,13322.000000,1.339600e+04
mean,7.236169,27.379367,1.035906,38.616874,3.373218e+17
std,1.328328,6.136034,0.198488,2.575000,5.194092e+18
min,0.562179,13.000000,1.000000,19.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-3.340563e+18
50%,7.319347,27.000000,1.000000,39.000000,-2.605984e+17
75%,8.062305,32.000000,1.000000,40.000000,5.896568e+18
max,17.813351,50.000000,4.000000,47.000000,8.599690e+18


In [11]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [12]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.125340,False,23,Single(1),31.0,3095933535584005890
1,5.249206,True,26,Single(1),37.0,3095933535584005890
2,6.624891,False,25,Single(1),37.0,3095933535584005890
3,6.867399,False,32,Single(1),39.0,3095933535584005890
4,6.876218,True,20,Single(1),39.0,3095933535584005890


In [13]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13391,6.750554,Unknown,34,Single(1),38.0,-774501970389208065
13392,7.810978,Unknown,41,Single(1),39.0,-774501970389208065
13393,4.687028,Unknown,22,Single(1),35.0,-774501970389208065
13394,7.063611,Unknown,25,Single(1),42.0,-774501970389208065
13395,8.935335,Unknown,36,Single(1),38.0,-774501970389208065


In [14]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26626.000000,26626.000000,26626.000000,2.662600e+04
mean,7.236250,27.377901,38.619695,3.382192e+17
std,1.326111,6.134908,2.566867,5.190836e+18
min,0.562179,13.000000,19.000000,-9.183606e+18
25%,6.563162,22.000000,38.000000,-3.340563e+18
50%,7.317142,27.000000,39.000000,-2.605984e+17
75%,8.062305,32.000000,40.000000,5.896568e+18
max,17.813351,50.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [15]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [16]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6358 eval.csv
  26626 train.csv
  32984 total
==> eval.csv <==
9.06320359082,False,37,Single(1),38.0,8904940584331855459
7.75365775454,False,18,Single(1),38.0,-7146494315947640619
8.1240343547,False,23,Single(1),40.0,6365946696709051755
8.68841774542,True,18,Single(1),41.0,-4740473290291881219
6.4992274837599995,False,20,Single(1),38.0,6365946696709051755
7.5618555866,True,27,Single(1),41.0,270792696282171059
6.9996768185,False,32,Single(1),36.0,8904940584331855459
7.63019888782,True,26,Single(1),40.0,1569531340167098963
7.31273323054,False,17,Single(1),38.0,-6141045177192779423
6.93794738514,False,21,Single(1),40.0,-1866590652208008467

==> train.csv <==
7.12534030784,False,23,Single(1),31.0,3095933535584005890
5.24920645822,True,26,Single(1),37.0,3095933535584005890
6.6248909731,False,25,Single(1),37.0,3095933535584005890
6.8673994613,False,32,Single(1),39.0,3095933535584005890
6.87621795178,True,20,Single(1),39.0,3095933535584005890
7.06361087448,True,20,Single(1),40.0,3095933535

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License